In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
data=pd.read_csv('/content/tweet_emotions.csv')

In [3]:
data.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [5]:
pd.set_option('display.max_colwidth',None)

In [7]:
messages=data[['sentiment','content']]

In [8]:
messages.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habit earlier and i started freakin at his part =[
1,sadness,Layin n bed with a headache ughhhh...waitin on your call...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,"@dannycastillo We want to trade with someone who has Houston tickets, but no one will."


Preprocessing

In [44]:
import gensim
messages['content_clean_gensim']=messages['content'].apply(lambda x:gensim.utils.simple_preprocess(x))
messages.head()

,sentiment,content,content_clean_gensim
0,empty,@tiffanylue i know i was listenin to bad habit earlier and i started freakin at his part =[,"[tiffanylue, know, was, listenin, to, bad, habit, earlier, and, started, freakin, at, his, part]"
1,sadness,Layin n bed with a headache ughhhh...waitin on your call...,"[layin, bed, with, headache, ughhhh, waitin, on, your, call]"
2,sadness,Funeral ceremony...gloomy friday...,"[funeral, ceremony, gloomy, friday]"
3,enthusiasm,wants to hang out with friends SOON!,"[wants, to, hang, out, with, friends, soon]"
4,neutral,"@dannycastillo We want to trade with someone who has Houston tickets, but no one will.","[dannycastillo, we, want, to, trade, with, someone, who, has, houston, tickets, but, no, one, will]"


In [52]:
#replace the content value with cleaned data
messages['content']=messages['content_clean_gensim']
messages.head()


,sentiment,content,content_clean_gensim
0,empty,"[tiffanylue, know, was, listenin, to, bad, habit, earlier, and, started, freakin, at, his, part]","[tiffanylue, know, was, listenin, to, bad, habit, earlier, and, started, freakin, at, his, part]"
1,sadness,"[layin, bed, with, headache, ughhhh, waitin, on, your, call]","[layin, bed, with, headache, ughhhh, waitin, on, your, call]"
2,sadness,"[funeral, ceremony, gloomy, friday]","[funeral, ceremony, gloomy, friday]"
3,enthusiasm,"[wants, to, hang, out, with, friends, soon]","[wants, to, hang, out, with, friends, soon]"
4,neutral,"[dannycastillo, we, want, to, trade, with, someone, who, has, houston, tickets, but, no, one, will]","[dannycastillo, we, want, to, trade, with, someone, who, has, houston, tickets, but, no, one, will]"


In [55]:
#feature extraction
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer()
x=vectorizer.fit_transform(messages['content'].apply(lambda x: ' '.join(x)))


In [56]:
messages.head()

,sentiment,content,content_clean_gensim
0,empty,"[tiffanylue, know, was, listenin, to, bad, habit, earlier, and, started, freakin, at, his, part]","[tiffanylue, know, was, listenin, to, bad, habit, earlier, and, started, freakin, at, his, part]"
1,sadness,"[layin, bed, with, headache, ughhhh, waitin, on, your, call]","[layin, bed, with, headache, ughhhh, waitin, on, your, call]"
2,sadness,"[funeral, ceremony, gloomy, friday]","[funeral, ceremony, gloomy, friday]"
3,enthusiasm,"[wants, to, hang, out, with, friends, soon]","[wants, to, hang, out, with, friends, soon]"
4,neutral,"[dannycastillo, we, want, to, trade, with, someone, who, has, houston, tickets, but, no, one, will]","[dannycastillo, we, want, to, trade, with, someone, who, has, houston, tickets, but, no, one, will]"


In [65]:
#model building
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, messages['sentiment'], test_size=0.2, random_state=42)

In [66]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier()
clf.fit(x_train,y_train)

RandomForestClassifier()

In [69]:
from sklearn import metrics
y_pred=clf.predict(x_train)
metrics.accuracy_score(y_pred,y_train)


0.99565625

In [70]:
import tensorflow as tf
timesteps=20
features=50
timesteps_encoder=30
timesteps_decoder=20

In [72]:
#simple RNN
model_rnn=tf.keras.Sequential([tf.keras.layers.SimpleRNN(64,activation='relu',input_shape=(timesteps,features)),
                               tf.keras.layers.Dense(10,activation='softmax')])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [73]:
#lstm
model_lstm=tf.keras.Sequential([tf.keras.layers.LSTM(64,activation='tanh',input_shape=(timesteps,features)),
                                tf.keras.layers.Dense(10,activation='softmax')])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [77]:
#gru
model_gru=tf.keras.Sequential([tf.keras.layers.GRU(64,activation='tanh',input_shape=(timesteps,features)),
                              tf.keras.layers.Dense(10,activation='softmax')])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
